<a href="https://colab.research.google.com/github/giuliocapecchi/IR_project/blob/main/IR_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#%pip install torch matplotlib nltk tqdm gdown ir_datasets humanize

# 1. Download and prepare the collection

In [1]:
# chosen_collection can be one of ["vaswani", "msmarco", "covid"]

chosen_collection = "msmarco"

In [2]:
import gdown
import ir_datasets
import pandas as pd
import os

if chosen_collection not in ["vaswani", "msmarco"]:
    raise ValueError("chosen_collection must be one of ['vaswani', 'msmarco']")

if chosen_collection == "msmarco":

    os.makedirs('./collection/msmarco', exist_ok=True)

    url_collection = 'https://drive.google.com/uc?id=1_wXJjiwdgc9Kpt7o7atP8oWe-U4Z56hn'
    
    if not os.path.exists('./collection/msmarco/msmarco.tsv'):
        gdown.download(url_collection, './collection/msmarco/msmarco.tsv', quiet=False)
    
    """os.makedirs('./pickles', exist_ok=True)
    if not os.path.exists('./pickles/stats.pkl'):
        gdown.download(url_stats, './pickles/stats.pkl', quiet=False)
    if not os.path.exists('./pickles/lex.pkl'):
        gdown.download(url_lex, './pickles/lex.pkl', quiet=False)
    if not os.path.exists('./pickles/inv.pkl'):
        gdown.download(url_inv, './pickles/inv.pkl', quiet=False)
    if not os.path.exists('./pickles/doc.pkl'):
        gdown.download(url_doc, './pickles/doc.pkl', quiet=False)"""

elif chosen_collection == "vaswani":
    os.makedirs('./collection/vaswani', exist_ok=True)

    vaswani_dataset = ir_datasets.load(chosen_collection)
    docs = list(vaswani_dataset.docs_iter())
    df = pd.DataFrame(docs)
    df['doc_id'] = (df['doc_id'].astype(int) - 1).astype(str)
    # rimuovi i \n da ogni documento
    df['text'] = df['text'].str.replace('\n', ' ')
    if not os.path.exists('./collection/vaswani/vaswani.tsv'):
        df.to_csv('./collection/vaswani/vaswani.tsv', sep='\t', header=False, index=False)

Standard preprocessing but with the usage of the *PyStemmer* library.

In [3]:
import re
import string
import nltk
import Stemmer # PyStemmer


nltk.download("stopwords", quiet=True)
STOPWORDS = set(nltk.corpus.stopwords.words("english"))
STEMMER = Stemmer.Stemmer('english')
# stemmer = nltk.stem.PorterStemmer().stem # much slower than PyStemmer


def preprocess(s):
    # lowercasing
    s = s.lower()
    # ampersand and special chars
    s = re.sub(r"[‘’´“”–-]", "'", s.replace("&", " and ")) # this replaces & with 'and' and normalises quotes
    # acronyms
    s = re.sub(r"\.(?!(\S[^. ])|\d)", "", s) # this removes dots that are not part of an acronym
    # remove punctuation
    s = s.translate(str.maketrans(string.punctuation, " " * len(string.punctuation)))
    # strip whitespaces
    s = re.sub(r"\s+", " ", s).strip()
    # tokenisation
    tokens = [t for t in s.split() if t not in STOPWORDS]
    # stemming
    return STEMMER.stemWords(tokens)

In [4]:
import time

def profile(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        ms = (end - start) * 1000
        print(f"{f.__name__} ({ms:.3f} ms)")
        return result
    return f_timer

In [5]:
import pickle
import humanize
import os
from tqdm import tqdm

def print_pickled_size(var_name, var):
    # If the 'tmp' directory does not exist, we first create it
    os.makedirs('./tmp', exist_ok=True)
    with open(f"./tmp/{var_name}.pickle", 'wb') as f:
        pickle.dump(var, f)
    print(f'{var_name} requires {humanize.naturalsize(os.path.getsize(f"./tmp/{var_name}.pickle"))}')
    os.remove(f"./tmp/{var_name}.pickle")
    os.removedirs('./tmp')


def vbyte_encode(number):
    bytes_list = bytearray()
    while True:
        byte = number & 0x7F # Prendi i 7 bit meno significativi -> 0111 1111 = 0x7F
        number >>= 7 # Shifta a destra di 7 bit
        if number:
            bytes_list.append(byte) # Aggiungo i 7 bit al risultato
        else:
            bytes_list.append(0x80 | byte) # Aggiungo i 7 bit con il bit di continuazione, 0x80 = 1000 0000
            break
    return bytes(bytes_list)

def vbyte_decode(bytes_seq):
    number = 0
    for i, byte in enumerate(bytes_seq):
        number |= (byte & 0x7F) << (7 * i)
        if byte & 0x80:
            break
    return number

def decode_concatenated_vbyte(encoded_bytes):
    decoded_numbers = []
    current_number = 0
    shift_amount = 0
    
    for byte in encoded_bytes:
        if byte & 0x80:  # Bit di continuazione trovato, fine del numero
            current_number |= (byte & 0x7F) << shift_amount
            decoded_numbers.append(current_number)
            current_number = 0
            shift_amount = 0
        else:  # Continuo a comporre il numero
            current_number |= (byte & 0x7F) << shift_amount
            shift_amount += 7
    
    return decoded_numbers

#------------------------------------------------------------------------------------------------------------------------------------------------------------------#

def compress_index(lexicon, inv_d, inv_f):    
    compressed_inv_d = {}
    compressed_inv_f = {}

    for term, (termid, df, _) in tqdm(lexicon.items(), desc="Compressing lists", unit="term"):
        encoded_d = bytearray()
        for x in inv_d[termid]:
            encoded_d.extend(vbyte_encode(x)) 
        assert decode_concatenated_vbyte(encoded_d) == inv_d[termid]
        compressed_inv_d[termid] = encoded_d

        encoded_f = bytearray()
        for x in inv_f[termid]:
            encoded_f.extend(vbyte_encode(x))
        assert decode_concatenated_vbyte(encoded_f) == inv_f[termid]
        compressed_inv_f[termid] = encoded_f

    return compressed_inv_d, compressed_inv_f

## Functions to build the inverted index

In [6]:
import pandas as pd
from collections import Counter
from tqdm.auto import tqdm

def build_index(filepath, batch_size=10000):
    total_documents = sum(1 for _ in open(filepath)) # get total number of documents

    lexicon = {}
    inv_d = {}
    inv_f = {}
    doc_index = []
    total_dl = 0
    num_docs = 0
    termid = 0

    with open(filepath, 'r') as file:        
        batch = []
        
        with tqdm(total=total_documents, desc="Processing documents", unit="doc") as pbar:
            for line in file:
                batch.append(line.strip())
                
                # when the batch is full, we process it
                if len(batch) >= batch_size:
                    for line in batch:
                        doc_id, text = line.split('\t', 1) # '1' specifies the number of splits
                        doc_id = int(doc_id)
                        tokens = preprocess(text)
                        token_tf = Counter(tokens)

                        for token, tf in token_tf.items():
                            if token not in lexicon:
                                lexicon[token] = [termid, 0, 0] # termid, df, tf
                                inv_d[termid], inv_f[termid] = [], [] # docids, freqs
                                termid += 1
                            token_id = lexicon[token][0]  # get termid
                            inv_d[token_id].append(doc_id)  # add doc_id to the list of documents containing the term
                            inv_f[token_id].append(tf)  # add term frequency for this doc
                            lexicon[token][1] += 1  # increment document frequency (df)
                            lexicon[token][2] += tf  # increment total term frequency (tf)

                        doclen = len(tokens)
                        doc_index.append((str(doc_id), doclen))
                        total_dl += doclen
                        num_docs += 1                    
                    # update progress bar for each processed document
                    pbar.update(len(batch))
                    batch = []

            # process the remaining documents in the last batch
            if batch:
                for line in batch:
                    doc_id, text = line.split('\t', 1)
                    doc_id = int(doc_id)
                    tokens = preprocess(text)
                    token_tf = Counter(tokens)

                    for token, tf in token_tf.items():
                        if token not in lexicon:
                            lexicon[token] = [termid, 0, 0]
                            inv_d[termid], inv_f[termid] = [], []
                            termid += 1
                        token_id = lexicon[token][0]  # get termid
                        inv_d[token_id].append(doc_id)  # get doc_id to the list of documents containing the term
                        inv_f[token_id].append(tf)  # get term frequency for this doc
                        lexicon[token][1] += 1  # increment document frequency (df)
                        lexicon[token][2] += tf  # increment total term frequency (tf)

                    doclen = len(tokens)
                    doc_index.append((str(doc_id), doclen))
                    total_dl += doclen
                    num_docs += 1                    
                    pbar.update(1)
                    
     # Calculate average document length (avdl)
    avdl = total_dl / num_docs if num_docs > 0 else 0
                    
    stats = {
        'num_docs': num_docs,
        'num_terms': len(lexicon),
        'num_tokens': total_dl,
        'avdl': avdl  # Add avdl to stats
    }
    return lexicon, {'docids': inv_d, 'freqs': inv_f}, doc_index, stats

In [7]:
import math
import bisect


class InvertedIndex:

    class PostingListIterator:
        def __init__(self, docids, freqs, doc, avdl):
            self.docids = docids
            self.freqs = freqs
            self.pos = 0
            self.doc = doc
            self.total_docs_number = len(doc)
            self.avdl = avdl

        def docid(self):
            if self.is_end_list():
                return math.inf
            return self.docids[self.pos]


        ###################################################################################        
        def score(self): # TODO : check if correct, this is TF-IDF score
            """
            Calculate TF-IDF score of the current document in the posting list.
            """
            if self.is_end_list():
                return math.inf 
            
            tf = self.freqs[self.pos]
                        
            if tf > 0:
                wtd = 1 + math.log(tf)
            else:
                wtd = 0 # avoid log(0)
            
            df = self.len()  # document frequency
            if df > 0:
                idf = math.log(self.total_docs_number / df)
            else:
                idf = 0  # avoid log(0)
            
            # finally calculate tf-idf score
            tfidf = wtd * idf

            return tfidf

        ###################################################################################
        # new score_bm25 function
        # TODO: check if correct
        def score_bm25(self): # Modified to match the BM25 formula from the slides
            if self.is_end_list():
                return math.inf
            else:
                # Standard BM25 parameters
                b = 0.75
                k_1 = 1.5
                
                # Length of the current document
                dl = self.doc[self.docid()][1]
                
                # Term frequency in the current document
                tf = self.freqs[self.pos]
                
                # Total number of documents in the collection
                N = self.total_docs_number
                
                # Number of documents containing the term
                n = self.len()  # document frequency
                
                # Calculate document length normalization component (B_j)
                B_j = (1 - b) + b * (dl / self.avdl)
                
                # Calculate the IDF component
                idf = math.log( N / n)
                
                # Calculate the BM25 score
                rsv_bm25 = ((tf) / (tf + k_1 * B_j)) * idf
                
                return rsv_bm25
            
            ###################################################################################

        def next(self, target=None):
            if not target:
                if not self.is_end_list():
                    self.pos += 1
            else:
                if target > self.docid():
                    self.pos = bisect.bisect_left(self.docids, target, self.pos)

        def is_end_list(self):
            return self.pos == len(self.docids)


        def len(self):
            return len(self.docids)
        

    def __init__(self, lex, inv, doc, stats):
        self.lexicon = lex
        self.inv = inv
        self.doc = doc
        self.stats = stats

    def num_docs(self):
        return self.stats['num_docs']
    
    def avdl(self):
        return self.stats['avdl']

    def get_posting(self, termid):
        return InvertedIndex.PostingListIterator(self.inv['docids'][termid], self.inv['freqs'][termid], self.doc, self.stats['avdl'])
    
    def get_termids(self, tokens):
        return [self.lexicon[token][0] for token in tokens if token in self.lexicon]

    def get_postings(self, termids):
        return [self.get_posting(termid) for termid in termids]
    

In [12]:
# import cProfile
# import pstats

# cProfile.run("build_index('./vaswani.tsv')", "output.prof")
# p = pstats.Stats("output.prof")
# p.sort_stats("cumtime").print_stats(10)
# os.remove("output.prof")

## Building the index on the chosen collection 

Now build up the index for the chosen collection. It is built only if a pickled version of its components doesn't exist already :

In [9]:
import pickle

# If the 'pickles' directory does not exist, we first create it
os.makedirs('./pickles', exist_ok=True)

if chosen_collection == "msmarco":
    try: # try to open the pickled files, else build the index
        with open('./pickles/inv_index.pkl', 'rb') as f:
            inv_index = pickle.load(f)
        
    except FileNotFoundError:
        lex, inv, doc, stats = build_index('./collection/'+chosen_collection + '/'+chosen_collection+'.tsv')

        # Save the lexicon, inverted lists, and document index to disk
        with open('./pickles/lex.pkl', 'wb') as f:
            pickle.dump(lex, f)
        with open('./pickles/inv.pkl', 'wb') as f:
            pickle.dump(inv, f)
        with open('./pickles/doc.pkl', 'wb') as f:
            pickle.dump(doc, f)
        with open('./pickles/stats.pkl', 'wb') as f:
            pickle.dump(stats, f)
                    
        # Compress the inverted lists
        #inv['docids'], inv['freqs'] = compress_index(lex, inv['docids'], inv['freqs'])
        
        inv_index = InvertedIndex(lex, inv, doc, stats)
        with open('./pickles/inv_index.pkl', 'wb') as f:
            pickle.dump(inv_index, f)
else:
    lex, inv, doc, stats = build_index('./collection/'+chosen_collection + '/'+chosen_collection+'.tsv')
    inv_index = InvertedIndex(lex, inv, doc, stats)


print(f"Numero di documenti: {inv_index.num_docs()}")

Numero di documenti: 8841823


In [12]:
# print avdl
print(f"Avdl: {inv_index.avdl()}")

Avdl: 34.687022122021666


In [15]:
# # Create a test collection
# test_collection = [
#     "0\tterm1 term2 term3",
#     "1\tterm2 term3 term4",
#     "2\tterm1 term4 term5",
#     "3\tterm3 term5 term6",
#     "4\tterm1 term2 term3",
# ]

# # Write the fake collection to a temporary file
# with open('./collection/test_collection.tsv', 'w') as f:
#     for line in test_collection:
#         f.write(line + '\n')

# # Call the build_index function on the test collection
# lex, inv, doc, stats = build_index('./collection/test_collection.tsv')

# # Print the results
# print("Lexicon:", lex)
# print("Inverted Index (docids):", inv['docids'])
# print("Inverted Index (freqs):", inv['freqs'])
# print("Document Index:", doc)
# print("Statistics:", stats)

# inv_index = InvertedIndex(lex, inv, doc, stats)

# query = "term1 term2"
# termids = inv_index.get_termids(query.split())
# postings = inv_index.get_postings(termids)
# print(f"Query: '{query}', TermID: {termids}")
# for posting in postings:
#     while not posting.is_end_list():
#         print(f"docid: {posting.docid()}, score: {posting.score_bm25()}")
#         posting.next()


In [16]:
#print_pickled_size('inv_index', inv_index)

# 2. Download and prepare queries

In [13]:
import gzip
import os

if chosen_collection not in ["vaswani", "msmarco"]:
    raise ValueError("chosen_collection must be one of ['vaswani', 'msmarco']")

if chosen_collection == "msmarco":
    if not os.path.exists('./collection/msmarco/msmarco-queries.tsv'):
        url = 'https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz'
        gdown.download(url, './collection/msmarco/msmarco-test2019-queries.tsv.gz', quiet=False)
        with gzip.open('./collection/msmarco/msmarco-test2019-queries.tsv.gz', 'rt') as f_in:
            with open('./collection/msmarco/msmarco-queries.tsv', 'w') as f_out:
                f_out.write(f_in.read())
        os.remove('./collection/msmarco/msmarco-test2019-queries.tsv.gz') # delete the compressed file
    queries = pd.read_csv('./collection/msmarco/msmarco-queries.tsv', sep='\t', header=None)
    queries.columns = ['qid', 'query']
    print("Number of queries: ",len(queries))

    if not os.path.exists('./collection/msmarco/msmarco-qrels.txt'):
        url = 'https://trec.nist.gov/data/deep/2019qrels-pass.txt'
        gdown.download(url, './collection/msmarco/msmarco-qrels.txt', quiet=False)
    qrels = pd.read_csv('./collection/msmarco/msmarco-qrels.txt', sep=' ', header=None)
    qrels.columns = ['qid', 'Q0', 'docid', 'rating']
    print("Number of relevance judgments: ",len(qrels))


elif chosen_collection == "vaswani":
    queries = pd.DataFrame(vaswani_dataset.queries_iter())
    queries.columns = ['qid', 'query']
    print("Number of queries: ",len(list(vaswani_dataset.queries_iter()))) 
    if not os.path.exists('./collection/vaswani/vaswani-queries.tsv'):
        queries.to_csv('./collection/vaswani/vaswani-queries.tsv', sep='\t', header=False, index=False)
    qrels = pd.DataFrame(vaswani_dataset.qrels_iter()) 
    qrels.columns = ['qid', 'docid', 'relevance', 'iteration']
    qrels['docid'] = (qrels['docid'].astype(int) - 1).astype(str) # convert to 0-based indexing

    if not os.path.exists('./collection/vaswani/vaswani-qrels.txt'):
        qrels.to_csv('./collection/vaswani/vaswani-qrels.txt', sep='\t', header=False, index=False)
    print("Number of relevance judgments: ",len(list(vaswani_dataset.qrels_iter())))

Number of queries:  200
Number of relevance judgments:  9260


In [14]:
from collections import namedtuple


class QueriesDataset:
    def __init__(self, df):
        self.queries = [Query(row.query_id, row.text) for row in df.itertuples()]

    def queries_iter(self):
        return iter(self.queries)

    def queries_count(self):
        return len(self.queries)
    
    def get_query(self, query_id):
        return self.queries[query_id]


Query = namedtuple('Query', ['query_id', 'text'])
queries.columns = ['query_id', 'text']
queries_dataset = QueriesDataset(queries)
print("The number of queries is: ", queries_dataset.queries_count())

The number of queries is:  200


Let's prepare the functions necessary to perform TAAT and DAAT query processing

First, we need a TopQueue class, which stores the top  K  (score, docid) tuples, using an heap 

In [15]:
import heapq

class TopQueue:
    def __init__(self, k=10, threshold=0.0):
        self.queue = []
        self.k = k
        self.threshold = threshold

    def size(self):
        return len(self.queue)

    def would_enter(self, score):
        return score > self.threshold

    def clear(self, new_threshold=None):
        self.queue = []
        if new_threshold:
            self.threshold = new_threshold

    def __repr__(self):
        return f'<{self.size()} items, th={self.threshold} {self.queue}'

    def insert(self, docid, score):
        if score > self.threshold:
            if self.size() >= self.k:
                heapq.heapreplace(self.queue, (score, docid))
            else:
                heapq.heappush(self.queue, (score, docid))
            if self.size() >= self.k:
                self.threshold = max(self.threshold, self.queue[0][0])
            return True
        return False

#print(sorted(topq.queue, reverse=True)) # print the queue sorted by score

### TAAT

In [16]:
from collections import defaultdict

def taat(postings, k=10):
    A = defaultdict(float)
    for posting in postings:
        current_docid = posting.docid()
        while current_docid != math.inf:
            A[current_docid] += posting.score()
            posting.next()
            current_docid = posting.docid()
    top = TopQueue(k)
    for docid, score in A.items():
        top.insert(docid, score)
    return sorted(top.queue, reverse=True)


def query_process(query, index):
    qtokens = set(preprocess(query))
    qtermids = index.get_termids(qtokens)
    postings = index.get_postings(qtermids)
    return taat(postings)

### DAAT

In [17]:
import math

def min_docid(postings):
    min_docid = math.inf
    for p in postings:
        if not p.is_end_list():
            min_docid = min(p.docid(), min_docid)
    return min_docid

def daat(postings, k=10):
    top = TopQueue(k)
    current_docid = min_docid(postings)
    while current_docid != math.inf:
        score = 0
        next_docid = math.inf
        for posting in postings:
            if posting.docid() == current_docid:
                score += posting.score()
                posting.next()
            if not posting.is_end_list():
                next_docid = posting.docid()
        top.insert(current_docid, score)
        current_docid = next_docid
    return sorted(top.queue, reverse=True)

def query_process(query, index):
    qtokens = set(preprocess(query))
    qtermids = index.get_termids(qtokens)
    postings = index.get_postings(qtermids)
    return daat(postings)

In [18]:
from tqdm import tqdm

@profile
def query_processing(queries_iter, fn):
    #for q in tqdm(queries_iter, desc="Processing queries", total=queries_dataset.queries_count(), unit="query"):
    for q in queries_iter:
        query = preprocess(q.text)
        termids = inv_index.get_termids(query)
        postings = inv_index.get_postings(termids)
        res = fn(postings)

In [23]:
# import cProfile
# import pstats

# cProfile.run("query_processing(queries_dataset.queries_iter(), taat)", "./perfm/result.prof")
# p = pstats.Stats("./perfm/result.prof")
# p.sort_stats("cumtime").print_stats(25)

In [ ]:
#query_processing(queries_dataset.queries_iter(), taat)

query_processing (285906.451 ms)


In [ ]:
#query_processing(queries_dataset.queries_iter(), daat)

query_processing (614704.556 ms)


---

# 3. Evaluation

A relevance assessment (called ***qrel*** in `ir_datasets`) is composed by:
* a **topic id** (called *query_id* in `ir_datasets`) as in a topic,
* a **docno** (called *doc_id* in `ir_datasets`) as in a document,
* a **judgement** (called *relevance* in `ir_datasets`) as a binary or graded relevance judgment/label, and
* an **iteration**, **UNUSED** and always equal to the string `'0'`.

In [ ]:
# get the qrels for the chosen collection
# as above but space separated
'''qrels = pd.read_csv('./collection/'+chosen_collection+'/'+chosen_collection+'-qrels.txt', sep=' ', header=None)
qrels.columns = ['query_id', 'doc_id', 'relevance', 'iteration']
print("Number of relevance judgments: ",len(qrels))'''

Number of relevance judgments:  9260


In [20]:
import os

def create_run_file(queries_iter, fn, run_id, output_file):
    """
    Preprocess the queries and write the results to a run file.
    :param queries_iter: Query iterator
    :param fn: Function to process the postings and return the results in the format (score, docid)
    :param run_id: Name identifier for the run
    :param output_file: Output run file
    """
    if not os.path.exists('./results'):
        os.makedirs('./results')
    with open(f"./results/{output_file}", "w") as f:
        for q in queries_iter:
            topic_id = q.query_id 
            query = preprocess(q.text)
            termids = inv_index.get_termids(query)
            postings = inv_index.get_postings(termids)
            results = fn(postings, k=100)
            
            if results:
                # Write results to the run file
                for rank, (score, docno) in enumerate(results, start=1):
                    line = f"{topic_id}\tQ0\t{docno}\t{rank}\t{score:.6f}\t{run_id}\n"
                    f.write(line)
            else:
                # Annotate that no results were found for this query
                line = f"{topic_id}\tQ0\tNO_RESULTS\t0\t0.0\t{run_id}\n"
                f.write(line)

    print(f"Run file {output_file} produced successfully.")

In [ ]:
#create_run_file(queries_dataset.queries_iter(), taat, "run", f"{chosen_collection}.run")

Run file msmarco.run produced successfully.


In [22]:
print(inv_index.num_docs())

8841823


In [23]:
# TODO docid == docno ?
# TODO adattare a ms marco
# open the qrel file and create a 'gold' dictionary that contains for each qid the list of relevant documents

# empty list for for the docids
'''rel_docids = []

gold = {}
with open('./collection/msmarco/msmarco-qrels.txt', 'r') as f:
    for line in f:
        qid, _, docid, rel = line.strip().split(' ')
        qid = int(qid)
        docid = int(docid)
        #if int(rel) > 0:
        if qid not in gold:
            gold[qid] = []
        gold[qid].append(docid)
        
        if docid not in rel_docids:
            rel_docids.append(docid)
        
    



# D = 0 ... num_docs-1 (0-based indexing)
# D = 

D = list(range(inv_index.num_docs())) # from 0 to number of documents
T = list(range(1,len(gold))) # from 1 to number of queries

# open the run file and create a 'results' dictionary that contains for each qid the list of retrieved documents
results = {}
with open('./results/msmarco.run', 'r') as f:
    for line in f:
        qid, _, docid, rank, score, _ = line.strip().split('\t')
        qid = int(qid)
        if docid == 'NO_RESULTS':
            docid = -1
        else:
            docid = int(docid)
        if qid not in results:
            results[qid] = []
        results[qid].append(docid)


A = {} # A will contain if the results returned by the IR system are actually relevant
for qid in gold:
    A[qid] = [1 if d in gold[qid] else 0 for d in D]



print("Number of queries in the qrel file: ", len(gold))
print("Number of queries in the run file: ", len(results))
print("Number of 'A': ", len(A))

# print("qrels:\t",gold)
# print("results:",results)
# # stampa i primi 10 elementi di A
# print("A:\t", {k: A[k] for k in list(A)[:10]})'''

'rel_docids = []\n\ngold = {}\nwith open(\'./collection/msmarco/msmarco-qrels.txt\', \'r\') as f:\n    for line in f:\n        qid, _, docid, rel = line.strip().split(\' \')\n        qid = int(qid)\n        docid = int(docid)\n        #if int(rel) > 0:\n        if qid not in gold:\n            gold[qid] = []\n        gold[qid].append(docid)\n        \n        if docid not in rel_docids:\n            rel_docids.append(docid)\n        \n    \n\n\n\n# D = 0 ... num_docs-1 (0-based indexing)\n# D = \n\nD = list(range(inv_index.num_docs())) # from 0 to number of documents\nT = list(range(1,len(gold))) # from 1 to number of queries\n\n# open the run file and create a \'results\' dictionary that contains for each qid the list of retrieved documents\nresults = {}\nwith open(\'./results/msmarco.run\', \'r\') as f:\n    for line in f:\n        qid, _, docid, rank, score, _ = line.strip().split(\'\t\')\n        qid = int(qid)\n        if docid == \'NO_RESULTS\':\n            docid = -1\n        e

In [24]:
# print T horizzontally
#print("T:\t", T)

In [ ]:
#print(results)

{1108939: [3652655, 2068541, 3647358, 5553584, 4502510, 3130232, 6707713, 6185573, 4928535, 5553585, 5840050, 4744533, 8596285, 6225297, 6166456, 3357552, 3899060, 1397503, 7919347, 7152561, 698106, 699259, 5471907, 141310, 8661642, 242056, 6388957, 6388955, 4772091, 4390191, 6980266, 6758075, 3985446, 3502557, 3357548, 631424, 753531, 3150726, 5807716, 1511466, 2251472, 371765, 8596283, 5163856, 2010859, 4049530, 4172981, 3745976, 6681205, 1979870, 6179026, 2011059, 1842680, 1027732, 457000, 456998, 456997, 6564869, 6715236, 8596284, 3097883, 1889170, 1851519, 6868009, 6041119, 935161, 8209350, 6521245, 5118561, 8256786, 6204916, 414222, 8363174, 5579740, 2834919, 7777453, 6969425, 5668417, 4772094, 3209267, 2918196, 2875828, 1640379, 582753, 8172843, 7844766, 7844765, 3865302, 3009955, 2115313, 917895, 610121, 8611344, 6959553, 6859965, 5992241, 5194301, 4612632, 4320195, 4034250], 1112389: [8138765, 5892677, 7817890, 4931198, 7120135, 7651152, 452842, 452841, 452838, 452837, 452836,

In [ ]:
#print(results[1108939])

[3652655, 2068541, 3647358, 5553584, 4502510, 3130232, 6707713, 6185573, 4928535, 5553585, 5840050, 4744533, 8596285, 6225297, 6166456, 3357552, 3899060, 1397503, 7919347, 7152561, 698106, 699259, 5471907, 141310, 8661642, 242056, 6388957, 6388955, 4772091, 4390191, 6980266, 6758075, 3985446, 3502557, 3357548, 631424, 753531, 3150726, 5807716, 1511466, 2251472, 371765, 8596283, 5163856, 2010859, 4049530, 4172981, 3745976, 6681205, 1979870, 6179026, 2011059, 1842680, 1027732, 457000, 456998, 456997, 6564869, 6715236, 8596284, 3097883, 1889170, 1851519, 6868009, 6041119, 935161, 8209350, 6521245, 5118561, 8256786, 6204916, 414222, 8363174, 5579740, 2834919, 7777453, 6969425, 5668417, 4772094, 3209267, 2918196, 2875828, 1640379, 582753, 8172843, 7844766, 7844765, 3865302, 3009955, 2115313, 917895, 610121, 8611344, 6959553, 6859965, 5992241, 5194301, 4612632, 4320195, 4034250]


In [26]:
'''def precision(results, qrels): # i qrels sono i relevance assesment files
    retrieved = len(results)
    retrieved_relevant = sum([qrels[d] for d in results])
    return retrieved_relevant/retrieved # effettivamente torna

def recall(results, qrels):
    relevant = sum(qrels)
    retrieved_relevant = sum([qrels[d] for d in results])
    return retrieved_relevant/relevant

def f1_score(results, qrels):
    P = precision(results, qrels)
    R = recall(results, qrels)
    return 2*R*P/(R+P)
    ''''''# TODO : rimuovi perchè non esiste che sia zero
    if P + R == 0:
        return 0'''
    
    
'''for t in T:
    # print results[t]
    # print A[t]
    print(results[t])
    print(A[t]) 

print(f'Topic {t}: F1 = {f1_score(results[t], A[t])}')
    
for t in T:
    print(f'Topic {t}: P = {precision(results[t], A[t])}')'''

"for t in T:\n    # print results[t]\n    # print A[t]\n    print(results[t])\n    print(A[t]) \n\nprint(f'Topic {t}: F1 = {f1_score(results[t], A[t])}')\n    \nfor t in T:\n    print(f'Topic {t}: P = {precision(results[t], A[t])}')"

In [27]:
'''import numpy as np

def precision_at_k(results, qrels, k):
    return precision(results[:k], qrels) # prende i primi k risultati e calcola la precisione su questi

for k in [1, 3, 5, 10, 20]:
    print(f'P@{k} = {np.mean([precision_at_k(results[t], A[t], k) for t in T])}') # media delle precisioni per ogni topic


def recall_at_k(results, qrels, k):
    return recall(results[:k], qrels)

for k in [1, 3, 5, 10, 20]:
    print(f'R@{k} = {np.mean([recall_at_k(results[t], A[t], k) for t in T])}') # stessa cosa di prima ma per recall'''

"import numpy as np\n\ndef precision_at_k(results, qrels, k):\n    return precision(results[:k], qrels) # prende i primi k risultati e calcola la precisione su questi\n\nfor k in [1, 3, 5, 10, 20]:\n    print(f'P@{k} = {np.mean([precision_at_k(results[t], A[t], k) for t in T])}') # media delle precisioni per ogni topic\n\n\ndef recall_at_k(results, qrels, k):\n    return recall(results[:k], qrels)\n\nfor k in [1, 3, 5, 10, 20]:\n    print(f'R@{k} = {np.mean([recall_at_k(results[t], A[t], k) for t in T])}') # stessa cosa di prima ma per recall"

# IR Measures

In [64]:
import ir_measures
from ir_measures import *

chosen_collection = "msmarco"

# Load the qrels
qrels = pd.read_csv('./collection/'+chosen_collection+'/'+chosen_collection+'-qrels.txt', sep=' ', header=None)
qrels.columns = ['query_id', 'Q0', 'doc_id', 'relevance']
qrels['query_id'] = qrels['query_id'].apply(str)
qrels['doc_id'] = qrels['doc_id'].apply(str)
qrels['relevance'] = qrels['relevance'].apply(int)
# Load the run
run_file = list(ir_measures.read_trec_run(f'./results/{chosen_collection}.run'))

In [56]:
# The Set Precision (SetP); i.e., the number of relevant docs divided by the total number retrieved
print(ir_measures.calc_aggregate([SetP], qrels, run_file))

# The Set Recall (SetR); i.e., the number of relevant docs divided by the total number of relevant documents
print(ir_measures.calc_aggregate([SetR], qrels, run_file))

{SetP: 0.26418604651162786}
{SetR: 0.34199089792047155}


In [ ]:
# PRECISION at k

print(ir_measures.calc_aggregate([P@5], qrels, run_file))
print(ir_measures.calc_aggregate([P(rel=1)@5], qrels, run_file))	# default is rel=1
print(ir_measures.calc_aggregate([P(rel=2)@5], qrels, run_file))
print(ir_measures.calc_aggregate([P(rel=3)@5], qrels, run_file))

print(ir_measures.calc_aggregate([P(rel=1, judged_only=True)@5], qrels, run_file))	# default is judged_only=False
print(ir_measures.calc_aggregate([P(rel=2, judged_only=True)@5], qrels, run_file))
print(ir_measures.calc_aggregate([P(rel=3, judged_only=True)@5], qrels, run_file))

{P@5: 0.5581395348837209}
{P@5: 0.5581395348837209}
{P(rel=2)@5: 0.39534883720930225}
{P(rel=3)@5: 0.17209302325581394}
{P@5: 0.5581395348837209}
{P(rel=2,judged_only=True)@5: 0.4372093023255814}
{P(rel=3,judged_only=True)@5: 0.19069767441860466}


In [58]:
# RECALL at k

print(ir_measures.calc_aggregate([R@100], qrels, run_file))
print(ir_measures.calc_aggregate([R(rel=1)@5], qrels, run_file))	# default is rel=1
print(ir_measures.calc_aggregate([R(rel=2)@5], qrels, run_file))
print(ir_measures.calc_aggregate([R(rel=3)@5], qrels, run_file))

print(ir_measures.calc_aggregate([R(rel=1, judged_only=True)@5], qrels, run_file))	# default is judged_only=False
print(ir_measures.calc_aggregate([R(rel=2, judged_only=True)@5], qrels, run_file))
print(ir_measures.calc_aggregate([R(rel=3, judged_only=True)@5], qrels, run_file))

{R@100: 0.34199089792047155}
{R@5: 0.06886452820043387}
{R(rel=2)@5: 0.09394562802992476}
{R(rel=3)@5: 0.09221170124734597}
{R(judged_only=True)@5: 0.0747698051731351}
{R(rel=2,judged_only=True)@5: 0.09949099373829699}
{R(rel=3,judged_only=True)@5: 0.09811780399852398}


In [57]:
print(ir_measures.calc_aggregate([P@5, P(rel=2)@5, nDCG@10, AP, AP(rel=2), Bpref, Bpref(rel=2), Judged@10], qrels, run_file))

{P(rel=2)@5: 0.39534883720930225, AP(rel=2): 0.1855357022003192, Judged@10: 0.7581395348837208, Bpref: 0.28740518536791293, AP: 0.20267250123025782, P@5: 0.5581395348837209, nDCG@10: 0.41352178810945667, Bpref(rel=2): 0.2345261473754138}


In [ ]:
# TODO : da rivedere

count = 0
for metric in ir_measures.iter_calc([P@5, P(rel=2)@5, nDCG@10, AP, AP(rel=2), Bpref, Bpref(rel=2), Judged@10], qrels, run_file):
  print(metric)
  count += 1
  if count >= 10: break # only show top 10 items

run_file_p_rel2_5 = {m.query_id: m.value for m in ir_measures.iter_calc([Bpref(rel=2)], qrels, run_file)}

from scipy.stats import ttest_rel
qids = list(run_file_p_rel2_5.keys())
#ttest_rel([run_file_p_rel2_5[v] for v in qids])


---

# Trying it out

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import time


# UI elements
search_bar = widgets.Text(
    placeholder='Type in a query...',
    description='Search:',
    layout=widgets.Layout(width='80%')
)
search_button = widgets.Button(
    description='Search',
    button_style='success',
    tooltip='Execute the query',
    icon='search'
)

score_function_rbtn = widgets.RadioButtons(options=['TF-IDF', 'BM25'], description='Scoring function:', disabled=False)
algo_rbtn = widgets.RadioButtons(options=['TAAT', 'DAAT'], description='Algorithm:', disabled=False)
_style = widgets.HTML(
    "<style>.widget-radio-box {flex-direction: row !important;}.widget-radio-box"
    " label{margin:2px !important;width: 100px !important;}</style>",
    layout=widgets.Layout(display="none"),
)

output_area = widgets.Output()


def on_search_click(b):
    with output_area:
        clear_output()  # clean previous output
        query = search_bar.value
        if not query.strip():
            print("Please, type in a query.")
            return
        
        selected_scoring_function = score_function_rbtn.value
        print(f"Selected scoring function: {selected_scoring_function}")
        # TODO : implement the selected scoring function

        selected_algorithm = algo_rbtn.value
        print(f"Selected Algorithm: {selected_algorithm}")
        
        start_time = time.time()
        # --- QUERY EXECUTION ---
        processed_query = preprocess(query)
        termids = inv_index.get_termids(processed_query)
        postings = inv_index.get_postings(termids)
        
        if selected_algorithm == 'TAAT':
            results = taat(postings)
        else:
            results = daat(postings)
        # ------------------------
        elapsed_time = (time.time() - start_time) * 1000 # convert in ms

        # finally show the results
        print(f"Found: {len(results)} documents\n")
        for res in results:
            res = (round(res[0], 4), res[1]) # TODO : si potrebbe spostare direttamente nella score function
            print(f" - {res}")
        print(f"\nExecution time: {elapsed_time:.2f} ms")

search_button.on_click(on_search_click)
# link search button to the enter key
search_bar.continuous_update = False
search_bar.observe(on_search_click, names='value')

top_row = widgets.HBox([score_function_rbtn,_style])
middle_row = widgets.HBox([algo_rbtn,_style])
bottom_row = widgets.HBox([search_bar, search_button])

# finally display the UI
display(widgets.VBox([top_row,middle_row, bottom_row]))
display(output_area)


# PyTerrier and Evaluation

In [59]:
import pyterrier as pt
import pandas as pd

# note that pt.started() and pt.init() are deprecated

if not pt.java.started():
    pt.java.init()

Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]


In [ ]:
'''# Load the pickled inverted index
df = pd.read_csv("collection/vaswani/vaswani.tsv", sep="\t")

# assign columns
df.columns = ["docno", "text"]

# Convert columns to strings
df["docno"] = df["docno"].astype(str)
df["text"] = df["text"].astype(str)

# Display the DataFrame to verify
print(df.head())'''

In [ ]:
'''indexer = pt.DFIndexer("./index_3docs", overwrite=True)
#indexer = pt.IterDictIndexer("./index_3docs")
indexref = indexer.index(df["text"], df["docno"])
indexref.toString()'''

In [61]:
indexref = pt.IndexRef.of(os.path.abspath("index_3docs/data.properties"))
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics())

Number of documents: 8841822
Number of terms: 1170682
Number of postings: 215238431
Number of fields: 1
Number of tokens: 288759502
Field names: [text]
Positions:   false



In [62]:
import pandas as pd
import re
import string


# File path for the input file
file_path = "collection/msmarco/msmarco-queries.tsv"  # Replace with your file path
# Read the input file
queries = pd.read_csv(file_path, sep="\t", header=None, names=["qid", "query"])

# Apply preprocessing to each query
queries["processed_query"] = queries["query"].apply(preprocess)

# Save the processed queries to a new file
output_path = "processed_queries.txt"
queries[["qid", "processed_query"]].to_csv(output_path, sep="\t", index=False)

# Read the processed file with the correct column names
processed_queries = pd.read_csv(output_path, sep="\t", names=["qid", "query"], skiprows=1)

In [70]:
import ast

def concatenate_list(query):
    """
    Funzione per concatenare gli elementi di una lista di termini in una stringa.
    """
    try:
        query_list = ast.literal_eval(query)  # Converte la stringa in lista
        return " ".join(query_list)  # Concatena gli elementi della lista con uno spazio
    except (ValueError, SyntaxError):
        return query  # Restituisci il valore originale se non è una lista valida


# Step 1: Leggi il file delle query originali
file_path = "collection/msmarco/msmarco-queries.tsv"  # Sostituisci con il percorso del file
queries = pd.read_csv(file_path, sep="\t", header=None, names=["qid", "query"])

# Step 2: Applica la funzione preprocess a tutte le query
queries["processed_query"] = queries["query"].apply(preprocess)

# Step 3: Salva le query preprocessate in un file temporaneo
temp_output_path = "processed_queries_temp.txt"
queries[["qid", "processed_query"]].to_csv(temp_output_path, sep="\t", index=False)

# Step 4: Rileggi il file temporaneo per applicare concatenate_list
processed_queries = pd.read_csv(temp_output_path, sep="\t")

# Applica concatenate_list alla colonna 'processed_query'
processed_queries["qid"] = processed_queries["qid"].astype(str)
processed_queries["query"] = processed_queries["processed_query"].apply(concatenate_list)
processed_queries = processed_queries[["qid", "query"]]
# Step 5: Salva il file finale con le colonne 'qid' e 'query'
final_output_path = "final_processed_queries.txt"
processed_queries[["qid", "query"]].to_csv(final_output_path, sep="\t", index=False)

# Output per conferma
print(f"File processato salvato in: {final_output_path}")
print(processed_queries.head())

processed_queries["qid"] = processed_queries["qid"].astype(str)


qrels = pd.read_csv('./collection/'+chosen_collection+'/'+chosen_collection+'-qrels.txt', sep=' ', header=None)
qrels.columns = ['qid', 'Q0', 'docno', 'relevance']
qrels['qid'] = qrels['qid'].apply(str)
qrels['docno'] = qrels['docno'].apply(str)
#qrels['relevance'] = qrels['relevance'].apply(int)

File processato salvato in: final_processed_queries.txt
       qid                  query
0  1108939        slow flow blood
1  1112389  counti grand rapid mn
2   792752                 ruclip
3  1119729           noseble nose
4  1105095  sugar lake lodg locat


In [72]:
queries_qids = set(processed_queries["qid"])
qrels_qids = set(qrels["qid"])

common_qids = queries_qids & qrels_qids
missing_in_qrels = queries_qids - qrels_qids
missing_in_queries = qrels_qids - queries_qids

# Mostra i risultati
print(f"Numero totale di qid in queries: {len(queries_qids)}")
print(f"Numero totale di qid in qrels: {len(qrels_qids)}")
print(f"Numero di qid comuni: {len(common_qids)}")
print(f"Qid mancanti nei qrels: {len(missing_in_qrels)}")
print(f"Qid mancanti nelle queries: {len(missing_in_queries)}")

Numero totale di qid in queries: 200
Numero totale di qid in qrels: 43
Numero di qid comuni: 43
Qid mancanti nei qrels: 157
Qid mancanti nelle queries: 0


In [73]:
import pandas as pd

# Percorso del file da leggere
input_file = "collection/msmarco/msmarco-qrels.txt"  # Sostituisci con il percorso del tuo file
output_file = "filtered_msmarco-qrels.txt"  # Nome del file di output

# Leggi il file con pandas
df = pd.read_csv(input_file, sep=" ", header=None, names=["qid", "Q0", "docno", "relevance"])

# Filtra le righe in cui la colonna 'relevance' è diversa da zero
filtered_df = df[df["relevance"] != 0]

# Salva il risultato in un nuovo file
filtered_df.to_csv(output_file, sep=" ", index=False, header=False)

print(f"File salvato in: {output_file}")

file_path = "filtered_msmarco-qrels.txt"

# Leggi il file con pandas
df = pd.read_csv(file_path, sep=" ", header=None, names=["qid", "Q0", "docno", "relevance"])

# Conta il numero di valori univoci nella colonna 'qid'
unique_values_count = df["qid"].nunique()

# Stampa il numero di valori univoci
print(f"Numero di valori univoci nella prima colonna ('qid'): {unique_values_count}")

"""queries_qids = set(processed_queries["qid"])
qrels_qids = set(qrels["qid"])

common_qids = queries_qids & qrels_qids
missing_in_qrels = queries_qids - qrels_qids
missing_in_queries = qrels_qids - queries_qids

# Mostra i risultati
print(f"Numero totale di qid in queries: {len(queries_qids)}")
print(f"Numero totale di qid in qrels: {len(qrels_qids)}")
print(f"Numero di qid comuni: {len(common_qids)}")
print(f"Qid mancanti nei qrels: {len(missing_in_qrels)}")
print(f"Qid mancanti nelle queries: {len(missing_in_queries)}")"""

File salvato in: filtered_msmarco-qrels.txt
Numero di valori univoci nella prima colonna ('qid'): 43


'queries_qids = set(processed_queries["qid"])\nqrels_qids = set(qrels["qid"])\n\ncommon_qids = queries_qids & qrels_qids\nmissing_in_qrels = queries_qids - qrels_qids\nmissing_in_queries = qrels_qids - queries_qids\n\n# Mostra i risultati\nprint(f"Numero totale di qid in queries: {len(queries_qids)}")\nprint(f"Numero totale di qid in qrels: {len(qrels_qids)}")\nprint(f"Numero di qid comuni: {len(common_qids)}")\nprint(f"Qid mancanti nei qrels: {len(missing_in_qrels)}")\nprint(f"Qid mancanti nelle queries: {len(missing_in_queries)}")'

In [74]:
file_path = "collection/msmarco/msmarco-qrels.txt"

df = pd.read_csv(file_path, sep=" ", header=None, names=["qid", "Q0", "docno", "relevance"])

# Conta il numero di valori univoci nella colonna 'qid'
unique_values_count = df["qid"].nunique()

# Stampa il numero di valori univoci
print(f"Numero di valori univoci nella prima colonna ('qid'): {unique_values_count}")

Numero di valori univoci nella prima colonna ('qid'): 43


In [75]:
# Definizione dei retriever
TF_IDF = pt.BatchRetrieve(index, wmodel="TF_IDF")
BM25 = pt.BatchRetrieve(index, wmodel="BM25")
PL2 = pt.BatchRetrieve(index, wmodel="PL2")

# Esegui l'esperimento
pt.Experiment(
    [TF_IDF, BM25, PL2],
    processed_queries,
    qrels,
    eval_metrics=["map", "recip_rank"]
)


C:\Users\jacop\AppData\Local\Temp\ipykernel_27108\3385254733.py:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  TF_IDF = pt.BatchRetrieve(index, wmodel="TF_IDF")
C:\Users\jacop\AppData\Local\Temp\ipykernel_27108\3385254733.py:3: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  BM25 = pt.BatchRetrieve(index, wmodel="BM25")
C:\Users\jacop\AppData\Local\Temp\ipykernel_27108\3385254733.py:4: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  PL2 = pt.BatchRetrieve(index, wmodel="PL2")


,name,map,recip_rank
0,TerrierRetr(TF_IDF),0.370491,0.774485
1,TerrierRetr(BM25),0.370611,0.774485
2,TerrierRetr(PL2),0.354771,0.769491


In [76]:
pt.Experiment(
    [TF_IDF, BM25, PL2],
    processed_queries,
    qrels,
    eval_metrics=["map"],
    round={"map" : 4},
    names=['TF-IDF', 'BM25', 'PL2'],
    baseline=0,
    correction='b'
)

,name,map,map +,map -,map p-value,map reject,map p-value corrected
0,TF-IDF,0.3705,NaN,NaN,NaN,False,NaN
1,BM25,0.3706,15.0,20.0,0.514755,False,1.000000
2,PL2,0.3548,6.0,35.0,0.000002,True,0.000004


In [77]:
pt.Experiment(
    [TF_IDF, BM25, PL2],
    processed_queries,
    qrels,
    eval_metrics=["map"],
    perquery=True
)

c:\Users\jacop\miniconda3\envs\mircv\lib\site-packages\pyterrier\pipelines.py:130: UserWarning: 157 topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.
  warn(f'{backfill_count} topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.')
c:\Users\jacop\miniconda3\envs\mircv\lib\site-packages\pyterrier\pipelines.py:130: UserWarning: 157 topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.
  warn(f'{backfill_count} topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.')
c:\Users\jacop\miniconda3\envs\mircv\lib\site-packages\pyterrier\pipelines.py:130: UserWarning: 157 topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.
  warn(f'{backfill_count} topic(s) not found in qrels. Scores for these topics are given as NaN and

,name,qid,measure,value
360,TerrierRetr(BM25),1005165,map,NaN
288,TerrierRetr(BM25),100983,map,NaN
363,TerrierRetr(BM25),101169,map,NaN
280,TerrierRetr(BM25),1012021,map,NaN
309,TerrierRetr(BM25),1014126,map,NaN
...,...,...,...,...
162,TerrierRetr(TF_IDF),953067,map,NaN
13,TerrierRetr(TF_IDF),962179,map,0.058898
150,TerrierRetr(TF_IDF),964223,map,NaN
61,TerrierRetr(TF_IDF),966413,map,NaN


In [78]:
pt.Experiment(
    [TF_IDF, BM25, PL2],
    processed_queries,
    qrels,
    eval_metrics=["map"],
    round={"map" : 4},
    names=['TF-IDF', 'BM25', 'PL2'],
    baseline=0,
    correction='b',
    save_dir='./'
)

,name,map,map +,map -,map p-value,map reject,map p-value corrected
0,TF-IDF,0.3705,NaN,NaN,NaN,False,NaN
1,BM25,0.3706,15.0,20.0,0.514755,False,1.000000
2,PL2,0.3548,6.0,35.0,0.000002,True,0.000004
